In [1]:
!pip install mlflow

  Using cached uvicorn-0.34.1-py3-none-any.whl.metadata (6.5 kB)
   ---------------------------------------- 0.0/28.2 MB ? eta -:--:--
   ---------------------------------------- 0.3/28.2 MB ? eta -:--:--
    --------------------------------------- 0.5/28.2 MB 1.2 MB/s eta 0:00:24
   - -------------------------------------- 0.8/28.2 MB 1.2 MB/s eta 0:00:23
   - -------------------------------------- 1.0/28.2 MB 1.2 MB/s eta 0:00:23
   - -------------------------------------- 1.3/28.2 MB 1.2 MB/s eta 0:00:23
   -- ------------------------------------- 1.6/28.2 MB 1.2 MB/s eta 0:00:22
   -- ------------------------------------- 1.8/28.2 MB 1.2 MB/s eta 0:00:22
   -- ------------------------------------- 2.1/28.2 MB 1.2 MB/s eta 0:00:22
   --- ------------------------------------ 2.4/28.2 MB 1.2 MB/s eta 0:00:22
   --- ------------------------------------ 2.6/28.2 MB 1.2 MB/s eta 0:00:22
   ---- ----------------------------------- 2.9/28.2 MB 1.2 MB/s eta 0:00:21
   ---- -----------------

In [2]:
from sklearn.datasets import load_diabetes
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from abc import ABC, abstractmethod
import mlflow
import mlflow.sklearn
import pandas as pd

class Evaluation(ABC):
    @abstractmethod
    def calculate_scores(self, y_true, y_pred):
        pass

class MSE(Evaluation):
    def calculate_scores(self, y_true, y_pred):
        return mean_squared_error(y_true, y_pred)

class R2(Evaluation):
    def calculate_scores(self, y_true, y_pred):
        return r2_score(y_true, y_pred)

# Load data
data = load_diabetes()
df = pd.DataFrame(data.data, columns=data.feature_names)
X_train, X_test, y_train, y_test = train_test_split(df, data.target, test_size=0.2, random_state=42)

# Train model
model = LinearRegression()
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)

# Evaluate
mse = MSE()
r2 = R2()
mse_score = mse.calculate_scores(y_test, y_pred)
r2_score_value = r2.calculate_scores(y_test, y_pred)

# MLflow logging
input_example = pd.DataFrame([X_test.iloc[0]])  # Now X_test is a DataFrame
mlflow.set_experiment("diabetes_evaluation")
mlflow.end_run()  # In case a run is already active

with mlflow.start_run():
    mlflow.log_metric("mse", mse_score)
    mlflow.log_metric("r2", r2_score_value)
    mlflow.sklearn.log_model(model, "model", input_example=input_example)
with mlflow.start_run():
    mlflow.log_metric("mse", mse_score)
    mlflow.log_metric("r2", r2_score_value)
    mlflow.sklearn.log_model(model, "model", input_example=input_example)

print("MSE Score:", mse_score)
print("R2 Score:", r2_score_value)



2025/04/15 16:04:15 INFO mlflow.tracking.fluent: Experiment with name 'diabetes_evaluation' does not exist. Creating a new experiment.


MSE Score: 2900.193628493482
R2 Score: 0.4526027629719195
